In [1]:
import numpy as np 

In [2]:
training_data = np.load("data/saved.npy")

In [3]:
training_data

array([[list([2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 1, 0, 0])],
       [list([0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 2, 0]),
        list([1, 0, 0, 0])],
       [list([4, 0, 4, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 1, 0])],
       ...,
       [list([32, 4, 2, 2, 2, 8, 64, 8, 16, 4, 32, 16, 4, 8, 4, 2]),
        list([0, 0, 1, 0])],
       [list([2, 32, 4, 4, 2, 8, 64, 8, 16, 4, 32, 16, 4, 8, 4, 2]),
        list([0, 0, 0, 1])],
       [list([2, 32, 8, 2, 2, 8, 64, 8, 16, 4, 32, 16, 4, 8, 4, 2]),
        list([0, 1, 0, 0])]], dtype=object)

In [4]:
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import median, mean
from collections import Counter

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
# import game env
from puzzle import GameGrid
env = GameGrid()
env.reset()

In [6]:
LR = 1e-2

In [7]:
def neural_network_model(input_size):

    network = input_data(shape=[None, input_size, 1], name='input')

    network = fully_connected(network, 200, activation='relu')
    #network = dropout(network, 0.8)

    network = fully_connected(network, 100, activation='relu')
    #network = dropout(network, 0.8)

    network = fully_connected(network, 4, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')

    return model

In [8]:
def train_model(training_data, model=False):

    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
    y = [i[1] for i in training_data]

    if not model:
        model = neural_network_model(input_size = len(X[0]))
    
    model.fit({'input': X}, {'targets': y}, n_epoch=2, batch_size=10, snapshot_step=10, show_metric=True, run_id='openai_learning')
    return model

In [9]:
model = train_model(training_data)

Training Step: 317637  | total loss: 1.39146 | time: 714.868s
| Adam | epoch: 002 | loss: 1.39146 - acc: 0.2126 -- iter: 1588180/1588188
Training Step: 317638  | total loss: 1.39074 | time: 714.872s
| Adam | epoch: 002 | loss: 1.39074 - acc: 0.2213 -- iter: 1588188/1588188
--


In [ ]:
num_games = 1000
goal_steps = 200
rewards = []
choices = []
accepted_scores = []

for each_game in range(num_games):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        #env.render()
        if len(prev_obs)==0:
            action = random.randrange(0,4)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1,len(prev_obs),1))[0]).item()
                
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        new_observation = np.array(np.array(new_observation).flatten().tolist())
        
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        
        score+=reward
        if done: break
        
    if each_game%100 == 0: print(each_game)
    rewards.append(score)
    accepted_scores.append(env.highest_score())

print('Average Score:',sum(rewards)/len(rewards))
print('choice 0:{} choice 1:{} choice 2:{} choice 3:{}'.format(choices.count(0)/len(choices),choices.count(1)/len(choices), choices.count(2)/len(choices), choices.count(3)/len(choices)))
print(Counter(accepted_scores))

0
100
200
300
400
500
600
700
